# QMzyme Serialization

In this example we follow the same steps from the [QMQM2 Calculation](https://qmzyme.readthedocs.io/en/latest/Examples/QMQM2%20Calculation.html) example, up until it is time to assign the calculation method. Let's pretend we got to that point and needed to read some literature to decide what level of theory we should use. So we will serialize the QMzymeModel object using the Python library [pickle](https://docs.python.org/3/library/pickle.html) by calling the method store_pickle, take a (hypothetical) break to read the literature, then reload the pickle file to continue.

#### Define the QM and QM2 regions

In [1]:
import QMzyme
from QMzyme.SelectionSchemes import DistanceCutoff
from QMzyme.data import PDB

QMzyme.data.residue_charges.update({'EQU': -1})
model = QMzyme.GenerateModel(PDB)
model.set_catalytic_center(selection='resid 263')
model.catalytic_center.set_charge(-1)
model.set_region(selection=DistanceCutoff, cutoff=5)
c_alpha_atoms = model.cutoff_5.get_atoms(attribute='name', value='CA')
model.cutoff_5.set_fixed_atoms(atoms=c_alpha_atoms)

/Users/hrk/anaconda3/envs/qmzyme/lib/python3.11/site-packages/MDAnalysis/topology/tpr/utils.py:51: DeprecationWarning: 'xdrlib' is deprecated and slated for removal in Python 3.13
  import xdrlib



Charge information not present. QMzyme will try to guess region charges based on residue names consistent with AMBER naming conventions (i.e., aspartate: ASP --> Charge: -1, aspartic acid: ASH --> Charge: 0.). See QMzyme.data.residue_charges for the full set.


#### Serialize QMzymeModel object 
The pickle file will by default be named after the QMzymeModel.name attribute, which by default is the base name of the file originally used to initialize the model. You can also specify a filename by passing the argument 'filename'. 

In [2]:
model.store_pickle()

#### Upload serialized QMzymeModel object

In [3]:
test = QMzyme.GenerateModel(pickle_file=f'{model.name}.pkl')

#### Define and assign calculation methods

In [4]:
qm1_method = QMzyme.QM_Method(basis_set='6-31+G**', 
               functional='wB97X-D3', 
               qm_input='OPT FREQ', 
               program='orca')

qm2_method = QMzyme.QM_Method(basis_set='6-31G*', 
               functional='wB97X-D3')

qm1_method.assign_to_region(region=model.catalytic_center)
qm2_method.assign_to_region(region=model.cutoff_5)

model.truncate()

QMzymeRegion cutoff_5 has an estimated charge of -2.
QMzymeRegion catalytic_center_cutoff_5_combined has an estimated charge of -2.

Truncated model has been created and saved to attribute 'truncated' and stored in QMzyme.CalculateModel.calculation under key QMQM2. This model will be used to write the calculation input.


#### Write calculation input file

The final step now is to use the write_input() method to create the calculation input file. When this method is called, the store_pickle() method is triggered and you will have the corresponding .pkl file for your QMzymeModel. This is especially useful because after your calculation is completed you can then load the calculation results back into the corresponding QMzymeRegion using the QMzymeRegion method store_calculation_results() passing it the calculation file. This method uses the cclib library to parse the output, so any calculation file that cclib can read can be stored to your QMzymeRegion.

In [5]:
model.write_input()

Use of this Writer class requires citing the following: 
 	1. Alegre‐Requena, J. V., Sowndarya S. V., S., Pérez‐Soto, R., Alturaifi, T. M. & Paton, R. S. AQME: Automated quantum mechanical environments for researchers and educators. WIREs Comput Mol Sci 13, e1663 (2023).
File /Users/hrk/git/QMzyme/docs/Examples/QCALC/catalytic_center_cutoff_5_combined_truncated.inp created.
